In [1]:
import os
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
# 为了直观地演示大型语料库中预训练词向量的语义，让我们将预训练词向量应用到词的相似性和类比任务中。

In [3]:
# 以下列出维度为50、100和300的预训练GloVe嵌入，可从GloVe网站下载。
# 预训练的fastText嵌入有多种语言。

In [4]:
#@save
d2l.DATA_HUB['glove.6b.50d'] = (d2l.DATA_URL + 'glove.6B.50d.zip',
                                '0b8703943ccdb6eb788e6f091b8946e82231bc4d')

#@save
d2l.DATA_HUB['glove.6b.100d'] = (d2l.DATA_URL + 'glove.6B.100d.zip',
                                 'cd43bfb07e44e6f27cbcc7bc9ae3d80284fdaf5a')

#@save
d2l.DATA_HUB['glove.42b.300d'] = (d2l.DATA_URL + 'glove.42B.300d.zip',
                                  'b5116e234e9eb9076672cfeabf5469f3eec904fa')

#@save
d2l.DATA_HUB['wiki.en'] = (d2l.DATA_URL + 'wiki.en.zip',
                           'c1816da3821ae9f43899be655002f6c723e91b88')

In [5]:
# 为了加载这些预训练的GloVe和fastText嵌入，我们定义了以下TokenEmbedding类。

In [6]:
#@save
class TokenEmbedding:
    """GloVe嵌入"""
    def __init__(self, embedding_name):
        self.idx_to_token, self.idx_to_vec = self._load_embedding(
            embedding_name)
        self.unknown_idx = 0
        self.token_to_idx = {token: idx for idx, token in
                             enumerate(self.idx_to_token)}

    def _load_embedding(self, embedding_name):
        idx_to_token, idx_to_vec = ['<unk>'], []
        data_dir = d2l.download_extract(embedding_name)
        # GloVe网站：https://nlp.stanford.edu/projects/glove/
        # fastText网站：https://fasttext.cc/
        with open(os.path.join(data_dir, 'vec.txt'), 'r', encoding='utf-8') as f:
            for line in f:
                elems = line.rstrip().split(' ')
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                # 跳过标题信息，例如fastText中的首行
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
        return idx_to_token, torch.tensor(idx_to_vec)

    def __getitem__(self, tokens):
        indices = [self.token_to_idx.get(token, self.unknown_idx)
                   for token in tokens]
        vecs = self.idx_to_vec[torch.tensor(indices)]
        return vecs

    def __len__(self):
        return len(self.idx_to_token)

In [7]:
glove_6b50d = TokenEmbedding('glove.6b.50d')

In [8]:
len(glove_6b50d)

400001

In [9]:
glove_6b50d.token_to_idx['thank'], glove_6b50d.idx_to_token[3367]

(5552, 'beautiful')

In [10]:
# 为了根据词向量之间的余弦相似性为输入词查找语义相似的词，我们实现了以下knn
# k邻近函数

In [11]:
def knn(W, x, k):
    # 增加1e-9以获得数值稳定性
    cos = torch.mv(W, x.reshape(-1,)) / (
        torch.sqrt(torch.sum(W * W, axis=1) + 1e-9) *
        torch.sqrt((x * x).sum())) # a * b / ||a|| ||b||
    _, topk = torch.topk(cos, k=k)
    return topk, [cos[int(i)] for i in topk]

In [12]:
# 然后，我们使用TokenEmbedding的实例embed中预训练好的词向量来搜索相似的词。

In [13]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.idx_to_vec, embed[[query_token]], k + 1)
    for i, c in zip(topk[1:], cos[1:]):  # 排除输入词
        print(f'{embed.idx_to_token[int(i)]}：cosine相似度={float(c):.3f}')

In [15]:
get_similar_tokens('student', 3, glove_6b50d)

teacher：cosine相似度=0.896
students：cosine相似度=0.882
teachers：cosine相似度=0.857


In [42]:
def get_analogy(token_a, token_b, token_c, embed):
    vecs = embed[[token_a, token_b, token_c]]
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.idx_to_vec, x,1)
    return embed.idx_to_token[int(topk[0])]  # 删除未知词v

In [41]:
get_analogy('man', 'woman', 'son', glove_6b50d)

'daughter'

In [31]:
get_analogy('beijing', 'china', 'tokyo', glove_6b50d)

'japan'

In [32]:
get_analogy('do', 'did', 'go', glove_6b50d)

'went'

In [33]:
get_analogy('king', 'man', 'queen', glove_6b50d)

'woman'